In [1]:
import numpy as np
import pandas as pd
import re
from tqdm import tqdm
pd.set_option('display.max_columns', None)

In [2]:
faers = pd.read_csv('/Users/architmatta/Documents/Columbia/Fall 2021/Capstone/Data/FAERS_2012Q4_2021Q1.csv', low_memory = False)

In [3]:
print(faers.shape)
faers.head()

(40584418, 22)


,primaryid,caseid,drug_seq,role_cod,drugname,val_vbm,route,dose_vbm,cum_dose_chr,cum_dose_unit,dechal,rechal,lot_nbr,exp_dt,nda_num,dose_amt,dose_unit,dose_form,dose_freq,origin,lot_num,prod_ai
0,34483284,3448328,1,PS,SUSTIVA,1,TRANSPLACENTAL,UNK UNK,NaN,NaN,NaN,U,NaN,NaN,020972,NaN,NaN,NaN,NaN,drug12q4.txt,NaN,NaN
1,34483284,3448328,2,SS,NEVIRAPINE,1,TRANSPLACENTAL,NaN,NaN,NaN,NaN,U,NaN,NaN,NaN,NaN,NaN,NaN,NaN,drug12q4.txt,NaN,NaN
2,34483284,3448328,3,SS,VIRACEPT,1,TRANSPLACENTAL,UNK UNK,NaN,NaN,NaN,U,NaN,NaN,NaN,NaN,NaN,NaN,NaN,drug12q4.txt,NaN,NaN
3,34483284,3448328,4,SS,COMBIVIR,1,TRANSPLACENTAL,UNK UNK,NaN,NaN,NaN,U,NaN,NaN,NaN,NaN,NaN,NaN,NaN,drug12q4.txt,NaN,NaN
4,34483284,3448328,5,SS,RETROVIR,1,TRANSPLACENTAL,UNK UNK,NaN,NaN,NaN,U,NaN,NaN,NaN,NaN,NaN,NaN,NaN,drug12q4.txt,NaN,NaN


In [4]:
faers_non_null = faers[faers['nda_num'].isnull() == False]
print(faers_non_null.shape)
faers_non_null.head()

(14274896, 22)


,primaryid,caseid,drug_seq,role_cod,drugname,val_vbm,route,dose_vbm,cum_dose_chr,cum_dose_unit,dechal,rechal,lot_nbr,exp_dt,nda_num,dose_amt,dose_unit,dose_form,dose_freq,origin,lot_num,prod_ai
0,34483284,3448328,1,PS,SUSTIVA,1,TRANSPLACENTAL,UNK UNK,NaN,NaN,NaN,U,NaN,NaN,020972,NaN,NaN,NaN,NaN,drug12q4.txt,NaN,NaN
5,35676563,3567656,1,PS,EXELON,1,ORAL,"1.5 MG, BID",59.714,MG,NaN,U,NaN,NaN,020823,1.5,MG,NaN,BID,drug12q4.txt,NaN,NaN
6,36059074,3605907,1,PS,EXELON,1,ORAL,"1.5 MG, BID",NaN,NaN,NaN,U,NaN,NaN,020823,1.5,MG,Capsule,BID,drug12q4.txt,NaN,NaN
7,36177452,3617745,1,PS,EXELON,1,ORAL,"1.5 MG, BID",NaN,NaN,NaN,U,NaN,NaN,020823,1.5,MG,Capsule,BID,drug12q4.txt,NaN,NaN
8,36177452,3617745,2,SS,EXELON,1,ORAL,"1.5 MG, ONCE/DIALY",NaN,NaN,NaN,U,NaN,NaN,020823,1.5,MG,Capsule,X1,drug12q4.txt,NaN,NaN


In [5]:
drug_nda = faers_non_null[['drugname', 'nda_num']]
print(drug_nda.shape)
#drug_nda = drug_nda.groupby('drugname')['nda_num'].agg(pd.Series.mode).reset_index()
drug_nda = drug_nda.groupby('drugname')['nda_num'].agg(lambda x: pd.Series.mode(x)[0]).reset_index()
print(drug_nda.shape)

(14274896, 2)
(69953, 2)


In [6]:
print(faers.drugname.nunique())
print(drug_nda.drugname.nunique())

573150
69953


In [7]:
drug_nda.head()

,drugname,nda_num
0,BUPRENORPHINE,018401
1,PROPOXYPHENE HYDROCHLORIDE AND ACETAMINOPHEN,040139
2,"ACTONEL (RISEDRONATE SODIUM) TABLET, UNKMG",20835
3,ALBENDAZOLE,20666.0
4,"ALBUTEROL SULFATE INHALATION SOLUTION, 0.083%",74880.0


In [8]:
del faers_non_null

In [9]:
drug_dict = drug_nda.set_index('drugname')['nda_num'].to_dict()

In [10]:
drugs_1 = set(faers.drugname.unique())
drugs_2 = set(drug_nda.drugname.unique())
drugs_left = list(drugs_1.difference(drugs_2))

In [11]:
for drug in drugs_left:
    drug_dict[drug] = None

In [12]:
column_list = faers['drugname'].tolist()
faers['nda_new'] = [drug_dict[i] for i in tqdm(column_list)]

100%|██████████| 40584418/40584418 [00:07<00:00, 5601401.66it/s]


In [13]:
'''
tqdm.pandas()
faers = pd.merge(faers, drug_nda, how = 'left', on = 'nda_num')
'''

"\ntqdm.pandas()\nfaers = pd.merge(faers, drug_nda, how = 'left', on = 'nda_num')\n"

In [14]:
'''
nda_new = []
for drug in faers.drugname:
    try:
        nda = drug_nda.loc[drug_nda['drugname'] == drug]['nda_num'].values[0]
        nda_new.append(nda)
    except:
        nda_new.append(None)
print(len(nda_new))
faers['nda_new'] = nda_new
'''

"\nnda_new = []\nfor drug in faers.drugname:\n    try:\n        nda = drug_nda.loc[drug_nda['drugname'] == drug]['nda_num'].values[0]\n        nda_new.append(nda)\n    except:\n        nda_new.append(None)\nprint(len(nda_new))\nfaers['nda_new'] = nda_new\n"

In [15]:
print(faers.shape)
faers.head()

(40584418, 23)


,primaryid,caseid,drug_seq,role_cod,drugname,val_vbm,route,dose_vbm,cum_dose_chr,cum_dose_unit,dechal,rechal,lot_nbr,exp_dt,nda_num,dose_amt,dose_unit,dose_form,dose_freq,origin,lot_num,prod_ai,nda_new
0,34483284,3448328,1,PS,SUSTIVA,1,TRANSPLACENTAL,UNK UNK,NaN,NaN,NaN,U,NaN,NaN,020972,NaN,NaN,NaN,NaN,drug12q4.txt,NaN,NaN,20972.0
1,34483284,3448328,2,SS,NEVIRAPINE,1,TRANSPLACENTAL,NaN,NaN,NaN,NaN,U,NaN,NaN,NaN,NaN,NaN,NaN,NaN,drug12q4.txt,NaN,NaN,020636
2,34483284,3448328,3,SS,VIRACEPT,1,TRANSPLACENTAL,UNK UNK,NaN,NaN,NaN,U,NaN,NaN,NaN,NaN,NaN,NaN,NaN,drug12q4.txt,NaN,NaN,20779.0
3,34483284,3448328,4,SS,COMBIVIR,1,TRANSPLACENTAL,UNK UNK,NaN,NaN,NaN,U,NaN,NaN,NaN,NaN,NaN,NaN,NaN,drug12q4.txt,NaN,NaN,20857.0
4,34483284,3448328,5,SS,RETROVIR,1,TRANSPLACENTAL,UNK UNK,NaN,NaN,NaN,U,NaN,NaN,NaN,NaN,NaN,NaN,NaN,drug12q4.txt,NaN,NaN,19655.0


In [16]:
print(sum(faers['nda_num'].isnull())/faers.shape[0])
print(sum(faers['nda_new'].isnull())/faers.shape[0])

0.648266583495173
0.0913347580837552


In [17]:
faers['time'] = faers['origin'].str[4:8].str.upper()

In [18]:
faers.head()

,primaryid,caseid,drug_seq,role_cod,drugname,val_vbm,route,dose_vbm,cum_dose_chr,cum_dose_unit,dechal,rechal,lot_nbr,exp_dt,nda_num,dose_amt,dose_unit,dose_form,dose_freq,origin,lot_num,prod_ai,nda_new,time
0,34483284,3448328,1,PS,SUSTIVA,1,TRANSPLACENTAL,UNK UNK,NaN,NaN,NaN,U,NaN,NaN,020972,NaN,NaN,NaN,NaN,drug12q4.txt,NaN,NaN,20972.0,12Q4
1,34483284,3448328,2,SS,NEVIRAPINE,1,TRANSPLACENTAL,NaN,NaN,NaN,NaN,U,NaN,NaN,NaN,NaN,NaN,NaN,NaN,drug12q4.txt,NaN,NaN,020636,12Q4
2,34483284,3448328,3,SS,VIRACEPT,1,TRANSPLACENTAL,UNK UNK,NaN,NaN,NaN,U,NaN,NaN,NaN,NaN,NaN,NaN,NaN,drug12q4.txt,NaN,NaN,20779.0,12Q4
3,34483284,3448328,4,SS,COMBIVIR,1,TRANSPLACENTAL,UNK UNK,NaN,NaN,NaN,U,NaN,NaN,NaN,NaN,NaN,NaN,NaN,drug12q4.txt,NaN,NaN,20857.0,12Q4
4,34483284,3448328,5,SS,RETROVIR,1,TRANSPLACENTAL,UNK UNK,NaN,NaN,NaN,U,NaN,NaN,NaN,NaN,NaN,NaN,NaN,drug12q4.txt,NaN,NaN,19655.0,12Q4


In [19]:
faers['drugnameclean'] = faers['drugname'].str.upper().str.strip().str.replace('\W+', '')

/Users/architmatta/opt/anaconda3/envs/capstone/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


In [20]:
faers['quarter'] = faers['time'].str[-1:]

In [21]:
faers.head()

,primaryid,caseid,drug_seq,role_cod,drugname,val_vbm,route,dose_vbm,cum_dose_chr,cum_dose_unit,dechal,rechal,lot_nbr,exp_dt,nda_num,dose_amt,dose_unit,dose_form,dose_freq,origin,lot_num,prod_ai,nda_new,time,drugnameclean,quarter
0,34483284,3448328,1,PS,SUSTIVA,1,TRANSPLACENTAL,UNK UNK,NaN,NaN,NaN,U,NaN,NaN,020972,NaN,NaN,NaN,NaN,drug12q4.txt,NaN,NaN,20972.0,12Q4,SUSTIVA,4
1,34483284,3448328,2,SS,NEVIRAPINE,1,TRANSPLACENTAL,NaN,NaN,NaN,NaN,U,NaN,NaN,NaN,NaN,NaN,NaN,NaN,drug12q4.txt,NaN,NaN,020636,12Q4,NEVIRAPINE,4
2,34483284,3448328,3,SS,VIRACEPT,1,TRANSPLACENTAL,UNK UNK,NaN,NaN,NaN,U,NaN,NaN,NaN,NaN,NaN,NaN,NaN,drug12q4.txt,NaN,NaN,20779.0,12Q4,VIRACEPT,4
3,34483284,3448328,4,SS,COMBIVIR,1,TRANSPLACENTAL,UNK UNK,NaN,NaN,NaN,U,NaN,NaN,NaN,NaN,NaN,NaN,NaN,drug12q4.txt,NaN,NaN,20857.0,12Q4,COMBIVIR,4
4,34483284,3448328,5,SS,RETROVIR,1,TRANSPLACENTAL,UNK UNK,NaN,NaN,NaN,U,NaN,NaN,NaN,NaN,NaN,NaN,NaN,drug12q4.txt,NaN,NaN,19655.0,12Q4,RETROVIR,4


In [22]:
faers['year'] = "20" + faers['time'].str[:2]

In [23]:
faers.head()

,primaryid,caseid,drug_seq,role_cod,drugname,val_vbm,route,dose_vbm,cum_dose_chr,cum_dose_unit,dechal,rechal,lot_nbr,exp_dt,nda_num,dose_amt,dose_unit,dose_form,dose_freq,origin,lot_num,prod_ai,nda_new,time,drugnameclean,quarter,year
0,34483284,3448328,1,PS,SUSTIVA,1,TRANSPLACENTAL,UNK UNK,NaN,NaN,NaN,U,NaN,NaN,020972,NaN,NaN,NaN,NaN,drug12q4.txt,NaN,NaN,20972.0,12Q4,SUSTIVA,4,2012
1,34483284,3448328,2,SS,NEVIRAPINE,1,TRANSPLACENTAL,NaN,NaN,NaN,NaN,U,NaN,NaN,NaN,NaN,NaN,NaN,NaN,drug12q4.txt,NaN,NaN,020636,12Q4,NEVIRAPINE,4,2012
2,34483284,3448328,3,SS,VIRACEPT,1,TRANSPLACENTAL,UNK UNK,NaN,NaN,NaN,U,NaN,NaN,NaN,NaN,NaN,NaN,NaN,drug12q4.txt,NaN,NaN,20779.0,12Q4,VIRACEPT,4,2012
3,34483284,3448328,4,SS,COMBIVIR,1,TRANSPLACENTAL,UNK UNK,NaN,NaN,NaN,U,NaN,NaN,NaN,NaN,NaN,NaN,NaN,drug12q4.txt,NaN,NaN,20857.0,12Q4,COMBIVIR,4,2012
4,34483284,3448328,5,SS,RETROVIR,1,TRANSPLACENTAL,UNK UNK,NaN,NaN,NaN,U,NaN,NaN,NaN,NaN,NaN,NaN,NaN,drug12q4.txt,NaN,NaN,19655.0,12Q4,RETROVIR,4,2012


In [38]:
faers['nda_new'] = faers['nda_new'].astype('float64').fillna(0).astype('int64').apply(lambda x: '{0:0>6}'.format(x))
faers['nda_new'] = "NDA" + faers['nda_new']

In [39]:
faers_summary = faers.groupby(['nda_new', 'year', 'quarter']).agg({'primaryid': 'count'})

In [40]:
faers_summary = faers_summary.reset_index()
faers_summary.columns = ['nda', 'year', 'quarter', 'adverse_affects']

In [41]:
faers_summary.tail()

,nda,year,quarter,adverse_affects
164548,NDA999999,2020,1,46601
164549,NDA999999,2020,2,39872
164550,NDA999999,2020,3,43889
164551,NDA999999,2020,4,44996
164552,NDA999999,2021,1,43432


In [43]:
faers_summary.to_csv('/Users/architmatta/Documents/Columbia/Fall 2021/Capstone/Data/FAERS_Summary_NDA.csv', index = False)

In [44]:
faers_summary.head()

,nda,year,quarter,adverse_affects
0,NDA-9223372036854775808,2015,1,1
1,NDA-9223372036854775808,2015,3,1
2,NDA-9223372036854775808,2016,4,1
3,NDA000000,2012,4,99940
4,NDA000000,2013,1,110566
